In [5]:
import os
import re
import numpy as np
from collections import defaultdict
from math import log

# Preprocessing function
def preprocess(text):
    return re.findall(r'\b\w+\b', text.lower())

# Load documents

def load_documents(folder_path):
    docs = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            with open(os.path.join(folder_path, filename), 'r', encoding='latin-1') as file: # Changed encoding to 'latin-1'
                docs[filename] = preprocess(file.read())
    return docs
# Load queries
def load_queries(query_file_path):
    with open(query_file_path, 'r') as file:
        return [line.strip() for line in file.readlines()]

# Compute term frequencies and document frequencies
def compute_statistics(docs):
    doc_count = len(docs)
    term_doc_freq = defaultdict(int)
    term_freq = defaultdict(lambda: defaultdict(int))

    for doc_id, words in docs.items():
        word_set = set(words)
        for word in words:
            term_freq[doc_id][word] += 1
        for word in word_set:
            term_doc_freq[word] += 1

    return term_freq, term_doc_freq, doc_count

# Compute relevance probabilities using BIM
def compute_relevance_prob(query, term_freq, term_doc_freq, doc_count):
    scores = {}
    for doc_id in term_freq:
        score = 1.0
        for term in query:
            tf = term_freq[doc_id].get(term, 0)
            df = term_doc_freq.get(term, 0)
            p_term_given_relevant = (tf + 1) / (sum(term_freq[doc_id].values()) + len(term_doc_freq))
            p_term_given_not_relevant = (df + 1) / (doc_count - df + len(term_doc_freq))
            score *= (p_term_given_relevant / p_term_given_not_relevant)
        scores[doc_id] = score
    return scores

# Main retrieval function
def retrieve_documents(folder_path, query_file_path):
    docs = load_documents(folder_path)
    queries = load_queries(query_file_path)

    term_freq, term_doc_freq, doc_count = compute_statistics(docs)

    for query in queries:
        query_terms = preprocess(query)
        scores = compute_relevance_prob(query_terms, term_freq, term_doc_freq, doc_count)
        ranked_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)
        print(f"Query: {query}")
        for doc_id, score in ranked_docs:
            print(f"Document: {doc_id}, Score: {score:.4f}")
        print()

# Example usage
folder_path = '/content/drive/MyDrive/Tech400_final_project'
query_file_path = '/content/drive/MyDrive/Query/query.txt'
retrieve_documents(folder_path, query_file_path)


Query: Land Rover Defender V8
Document: Land Rover Defender V8 .txt, Score: 1.1968
Document: Range Rover Sport SVR .txt, Score: 0.0184
Document: Dodge Challenger SRT Hellcat.txt, Score: 0.0049
Document: Jeep Grand Cherokee Trackhawk .txt, Score: 0.0049
Document: Ford Mustang Shelby GT500 .txt, Score: 0.0048
Document: Porsche Cayenne Turbo .txt, Score: 0.0047
Document: Audi RS Q8 .txt, Score: 0.0046
Document: Mercedes-AMG GLE 63 S .txt, Score: 0.0046
Document: Mercedes-AMG GT .txt, Score: 0.0045
Document: Chevrolet Corvette Z06 .txt, Score: 0.0045
Document: BMW M5 .txt, Score: 0.0044
Document: Jaguar F-PACE SVR .txt, Score: 0.0043
Document: Lexus LX 600 .txt, Score: 0.0026
Document: Tesla Model S Plaid .txt, Score: 0.0023
Document: Volkswagen Touareg R .txt, Score: 0.0023
Document: Acura MDX .txt, Score: 0.0022
Document: Ford Explorer .txt, Score: 0.0022
Document: Porsche 911 Turbo S .txt, Score: 0.0022
Document: Chrysler Pacifica .txt, Score: 0.0021
Document: Volvo XC90 .txt, Score: 0.

In [8]:
import random

import random

# Function to assign random relevance scores
def assign_random_relevance(queries, documents, relevance_scale=(0, 1)):
    relevance_scores = {}

    for query in queries:
        relevance_scores[query] = {}  # Use the query string directly, no tuple or list
        for doc in documents:
            # Assign a random relevance score between relevance_scale (0 and 1 by default)
            relevance_scores[query][doc] = random.randint(relevance_scale[0], relevance_scale[1])

    return relevance_scores

# Function to save relevance scores to a file
def save_relevance_scores_to_file(relevance_scores, output_file):
    with open(output_file, 'w') as f:
        for query, doc_scores in relevance_scores.items():
            for doc, score in doc_scores.items():
                f.write(f"{query},{doc},{score}\n")  # Write in the format query,document,score


# Example usage:
folder_path = '/content/drive/MyDrive/Tech400_final_project'
query_file_path = '/content/drive/MyDrive/Query/query.txt'
# Load documents and queries
documents = load_documents(folder_path)  # This returns a dictionary of doc_id -> content
queries = load_queries(query_file_path)  # This returns a list of queries

# Randomly assign relevance scores (0 for irrelevant, 1 for relevant)
random_relevance_scores = assign_random_relevance(queries, documents.keys())

# Save the relevance scores to query_relevance_score.txt
output_file = '/content/drive/MyDrive/Query/query_score.txt'
save_relevance_scores_to_file(random_relevance_scores, output_file)

print(f"Relevance scores saved to {output_file}")


Relevance scores saved to /content/drive/MyDrive/Query/query_score.txt
